source: https://www.kaggle.com/cfpb/us-consumer-finance-complaints

In [1]:
import pandas as pd
import os
import sys

In [2]:
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [3]:
feature_cols=["product", "sub_product", "issue", "sub_issue", "state", "zipcode", "company", "company_response_to_consumer", "timely_response", "consumer_disputed?", "consumer_complaint_narrative"]


In [67]:
df = pd.read_csv('../data/29Feb/consumer_complaints_kaggle.csv', usecols=feature_cols)

/Users/i854694/.virtualenvs/pipelines/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [68]:
df = df.rename({'consumer_disputed?': 'consumer_disputed', 'zipcode':'zip_code', 
                'company_response_to_consumer': 'company_response'}, axis=1)

In [69]:
df.head()

,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company,state,zip_code,company_response,timely_response,consumer_disputed
0,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,U.S. Bancorp,CA,95993,Closed with explanation,Yes,Yes
1,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,Wells Fargo & Company,CA,91104,Closed with explanation,Yes,Yes
2,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,Wells Fargo & Company,NY,11764,Closed with explanation,Yes,No
3,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,"Navient Solutions, Inc.",MD,21402,Closed with explanation,Yes,Yes
4,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,Resurgent Capital Services L.P.,GA,30106,Closed with explanation,Yes,Yes


In [7]:
len(df)

555957

In [8]:
df['consumer_complaint_narrative'].notnull().sum()

66806

In [43]:
df = df[df['consumer_complaint_narrative'].notnull()]

In [44]:
df['consumer_disputed'].value_counts()

No     51229
Yes    15577
Name: consumer_disputed, dtype: int64

In [11]:
df['c'] = df['consumer_disputed'].map({'Yes': 1, 'No': 0})
df = df.drop('consumer_disputed', axis=1)
df = df.rename(columns={"c": "consumer_disputed"})

In [36]:
df_top = df[:300000]

In [37]:
df_top['consumer_disputed'].value_counts()

0    239894
1     60106
Name: consumer_disputed, dtype: int64

In [51]:
df_majority = df[df['consumer_disputed']==0]
df_minority = df[df['consumer_disputed']==1]
df_majority = df_majority.sample(n=15577, replace=False)
df_resampled = pd.concat([df_majority, df_minority])
df_resampled = df_resampled.sample(frac=1).reset_index(drop=True)


In [52]:
df_resampled.head()

,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company,state,zip_code,company_response,timely_response,consumer_disputed
0,Bank account or service,Other bank product/service,"Account opening, closing, or management",NaN,My ex-husband opened an account for our XXXX c...,Wells Fargo & Company,CA,959XX,Closed with explanation,Yes,0
1,Debt collection,Credit card,Cont'd attempts collect debt not owed,Debt resulted from identity theft,"I was a victim of identity theft, I have alrea...",ROCKY POINT CAPITAL LLC,VA,222XX,Untimely response,No,0
2,Consumer Loan,Vehicle lease,Managing the loan or lease,NaN,I have been leasing a XXXX Chevrolet Volt thro...,Ally Financial Inc.,CA,940XX,Closed with explanation,Yes,0
3,Credit reporting,NaN,Incorrect information on credit report,Information is not mine,XXXX is reporting erroneous on my credit repor...,Equifax,NY,133XX,Closed with explanation,Yes,0
4,Credit reporting,NaN,Incorrect information on credit report,Account status,XXXX XXXX offers help in conjunction with the ...,Experian,GA,312XX,Closed with explanation,Yes,1


In [53]:
len(df_resampled)

31154

In [54]:
df_resampled.to_csv('../data/29Feb/cc_balanced_narrative.csv', index=False)

In [ ]:
# todo clean zip code

In [70]:
df['zip_code'].value_counts()

300XX    1205
48382     927
750XX     898
770XX     845
900XX     845
         ... 
50439       1
02070       1
47020       1
54245       1
03771       1
Name: zip_code, Length: 27052, dtype: int64

In [71]:
df['zip_code'] = df['zip_code'].str.replace('X', '0', regex=True)

In [91]:
df['zip_code'] = df['zip_code'].str.replace(r'\[|\*|\+|\-|`|\.|\ |\$|\/|!|\(', '0', regex=True)

In [92]:
df['zip_code'] = df['zip_code'].fillna(0)

In [93]:
df['zip_code'] = df['zip_code'].astype('int32')

In [65]:
df['sub_issue'].isna().sum()

33874

In [66]:
df['timely_response'].value_counts()

Yes    64638
No      2168
Name: timely_response, dtype: int64

In [94]:
df.tail()

,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company,state,zip_code,company_response,timely_response,consumer_disputed
555952,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,Bank of America,NaN,0,Closed with explanation,Yes,No
555953,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,Residential Credit Solutions,NaN,0,Closed with explanation,No,No
555954,Mortgage,Conventional fixed mortgage,"Loan modification,collection,foreclosure",NaN,NaN,BB&T Financial,NaN,0,Closed with explanation,Yes,No
555955,Debt collection,I do not know,Communication tactics,Frequent or repeated calls,NaN,Global Recovery Group,NaN,0,Untimely response,No,No
555956,Debt collection,I do not know,Cont'd attempts collect debt not owed,Debt is not mine,NaN,"CCS Financial Services, Inc.",NaN,0,Closed with explanation,Yes,No


In [ ]:
df = df.sample(frac=1, replace=False).reset_index(drop=True)

In [47]:
df.to_csv('../data/6Mar/consumer_complaints_with_narrative.csv', index=False)